In [ ]:
import random
import math

class treeNode():
    def __init__(self, board, parent):
        self.board = board

        if self.board.is_win() or self.board.is_draw():
            self.Terminal = True
        else:
            self.Terminal = False
        
        self.is_fully_expanded = self.Terminal 
        self.parent = parent
        self.visits = 0
        self.score = 0
        self.children = {}

class MCTS():
    def search(self, initial_state):
        self.root = treeNode(initial_state, None)

        for iteration in range(800):
            node = self.select(self.root)
            score = self.rollout(node.board)

            self.backpropagate(node, score)
        
        try:
            return self.get_best_move(self.root, 0)
        except:
            pass
    
    def select(self, node):
        while not node.Terminal:
            if node.is_fully_expanded:
                node = self.get_best_move(node, 2)
            else:
                return self.expand(node)
        return node
    
    def expand(self, node):
        states = node.board.generate_states()

        for state in states:
            if str(state.position) not in node.children:
                new_Node = treeNode(state, node)
                node.children[str(state.position)] = new_Node
            
                if len(states) == len(node.children):
                    node.is_fully_expanded = True
                return new_Node

        print("Should not come here!!")



    def rollout(self, board):
        while not board.is_win():
            try:
                board = random.choice(board.generate_states())
            except:
                return 0
        
        if board.player_2 == 'x': return 1
        elif board.player_2 == 'o': return -1 


    def backpropagate(self, node, score):
        while node is not None:
            node.visits += 1

            node.score += score
            node = node.parent


    def get_best_move(self, node, exploration_constant):
        best_score = float('-inf')
        best_moves = []

        for child_node in node.children.values():
            if child_node.board.player_2 == 'x' : current_player = 1
            elif child_node.board.player_2 == 'o' : current_player = -1

            move_score = current_player * child_node.score / child_node.visits + exploration_constant * math.sqrt(math.log(node.visits/child_node.visits))

            if move_score > best_score:
                best_score = move_score
                best_moves = [child_node]
            
            elif move_score == best_score:
                best_moves.append(child_node)

        return random.choice(best_moves)

In [ ]:
import streamlit as st
import time
import numpy as np
import pandas as pd
from copy import deepcopy

class Board():
    def __init__(self, board=None):
        self.player_1 = 'x'
        self.player_2 = 'o'
        self.empty_square = '~'
        self.position = {}
        self.init_board()
        if board is not None:
            self.__dict__ = deepcopy(board.__dict__)

    def init_board(self):
        for row in range(3):
            for col in range(3):
                self.position[row, col] = self.empty_square

    def make_move(self, row, col):
        board = Board(self)
        board.position[row, col] = self.player_1
        (board.player_1, board.player_2) = (board.player_2, board.player_1)
        return board

    def is_draw(self):
        for row, col in self.position:
            if self.position[row, col] == self.empty_square:
                return False
        return True

    def is_win(self):
        for col in range(3):
            if all(self.position[row, col] == self.player_2 for row in range(3)):
                return True
        for row in range(3):
            if all(self.position[row, col] == self.player_2 for col in range(3)):
                return True
        if all(self.position[i, i] == self.player_2 for i in range(3)):
            return True
        if all(self.position[i, 2 - i] == self.player_2 for i in range(3)):
            return True
        return False

    def generate_states(self):
        actions = []
        for row in range(3):
            for col in range(3):
                if self.position[row, col] == self.empty_square:
                    actions.append(self.make_move(row, col))
        return actions

def main():
    st.title("Tic Tac Toe Game - AI vs Human")
    if "play_first" not in st.session_state:
        st.session_state.play_first = None

    # Page 1: Ask if the user wants to play first
    if st.session_state.play_first is None:
        st.write("Would you like to play first? First player plays as X.")
        col1, col2 = st.columns(2)
        with col1:
            if st.button("Yes"):
                initialize_game(True)
        with col2:
            if st.button("No"):
                initialize_game(False)

    # Page 2: Tic Tac Toe game
    if st.session_state.play_first is not None:
        game_board()

def initialize_game(play_first):
    # Initialize game state based on the user's choice
    st.session_state.play_first = play_first
    st.session_state.user_symbol = 'x' if play_first else 'o'
    st.session_state.ai_symbol = 'o' if play_first else 'x'
    st.session_state.board = Board()
    st.session_state.current_player = st.session_state.user_symbol

    if not play_first:  # AI makes the first move if the user doesn't want to play first
        ai_first_move()

def ai_first_move():
    # AI makes the first move
    mcts = MCTS()
    best_move = mcts.search(st.session_state.board)
    st.session_state.board = best_move.board
    st.session_state.current_player = st.session_state.user_symbol  # Switch to user's turn

def game_board():
    # Display the game board
    board_placeholder = st.empty()
    status_placeholder = st.empty()
    
    st.markdown("""
        <style>
            .tic-tac-toe-table {
                width: 150px;  /* Adjust the width */
                height: 80px; /* Adjust the height */
                font-size: 25px; /* Adjust the font size of the table */
            }
            .tic-tac-toe-table td {
                border: 1px solid black;
                text-align: center; /* Optional: add border */
            }
        </style>
        """, unsafe_allow_html=True)

    # Modify your table rendering code to use the custom class
    def display_board():
        board = st.session_state.board
        board_table = [board.position[row, col] for row in range(3) for col in range(3)]
        df_board = pd.DataFrame(np.array(board_table).reshape(3, 3))
        table_html = df_board.to_html(classes='tic-tac-toe-table', header=False, index=False)
        board_placeholder.markdown(table_html, unsafe_allow_html=True)
    display_board()

    # Handle user's turn
    if st.session_state.current_player == st.session_state.user_symbol:
        status_placeholder.text("Your turn! Choose a square and press 'Play' to confirm.")
        st.session_state.selected_cell = st.selectbox("Choose a cell (1-9):", range(1, 10))
        if st.button("Play"):
            row = (st.session_state.selected_cell - 1) // 3
            col = (st.session_state.selected_cell - 1) % 3

            if st.session_state.board.position[row, col] == st.session_state.board.empty_square:
                st.session_state.board = st.session_state.board.make_move(row, col)
                display_board()
                if st.session_state.board.is_win():
                    status_placeholder.success("You win!")
                    st.button("Play Again", on_click=reset_game)
                    st.stop()
                elif st.session_state.board.is_draw():
                    status_placeholder.info("It's a draw!")
                    st.button("Play Again", on_click=reset_game)
                    st.stop()
                else:
                    st.session_state.current_player = st.session_state.ai_symbol
            else:
                st.warning('Illegal Move! Please try a different square.', icon="⚠️")

    # Handle AI's turn
    if st.session_state.current_player == st.session_state.ai_symbol:
        status_placeholder.text("AI's turn...")
        time.sleep(3)  # Simulate AI thinking for 3 seconds
        mcts = MCTS()
        best_move = mcts.search(st.session_state.board)
        st.session_state.board = best_move.board
        display_board()
        if st.session_state.board.is_win():
            status_placeholder.success("AI wins!")
            st.button("Play Again", on_click=reset_game)
            st.stop()
        elif st.session_state.board.is_draw():
            status_placeholder.info("It's a draw!")
            st.button("Play Again", on_click=reset_game)
            st.stop()
        else:
            st.session_state.current_player = st.session_state.user_symbol  # Switch back to user's turn
            status_placeholder.text("Your turn! Choose a square and press 'Play' to confirm.")

def reset_game():
    st.session_state.play_first = None
    st.session_state.user_symbol = None
    st.session_state.ai_symbol = None
    st.session_state.board = None
    st.session_state.current_player = None
    st.session_state.selected_cell = None

if __name__ == "__main__":
    main()
